In [6]:
from ultralytics import YOLO
import cv2
import numpy as np

# YOLO 모델 로드
model = YOLO('yolov8n-pose.pt')
cap = cv2.VideoCapture(0)

# 동물 얼굴 이미지 로드
img2 = cv2.imread('../data/panda.png')

def apply_animal_face(frame, keypoints, img2):
    try:
        org = keypoints[0]
        right_ear = keypoints[3]
        left_ear = keypoints[4]

        # 동물 얼굴 이미지 크기 조정
        resize_cal = int(abs(right_ear[0] - left_ear[0])*1.4)
        img2_resized = cv2.resize(img2, (resize_cal, resize_cal), interpolation=cv2.INTER_LINEAR)
        img2H, img2W, _ = img2_resized.shape

        # 위치 계산
        x_offset = org[0] - img2W // 2
        y_offset = org[1] - int(img2H * 0.7)

        # ROI 설정
        roi = frame[y_offset:y_offset + img2H, x_offset:x_offset + img2W]

        # 동물 얼굴 이미지 그레이스케일 및 마스크 생성
        img2_gray = cv2.cvtColor(img2_resized, cv2.COLOR_BGR2GRAY)
        mask_inv = cv2.bitwise_not(img2_gray)
        fg = cv2.bitwise_or(img2_resized, img2_resized, mask=mask_inv)

        # ROI와 동물 얼굴 이미지 결합
        final_roi = cv2.bitwise_or(roi, fg)
        frame[y_offset:y_offset + img2H, x_offset:x_offset + img2W] = final_roi

    except Exception as e:
        print(f"Error: {e}")

while True:
    ret, frame = cap.read()
    if not ret:
        print('ERROR: frame error')
        break

    frame = cv2.resize(frame, (720, 900))
    results = model(frame, stream=True)

    for r in results:
        keypoints = r.keypoints

        for ks in keypoints:
            k = np.array(ks.xy[0].cpu(), dtype=int)
            apply_animal_face(frame, k, img2)

    cv2.imshow('result preview', frame)
    key = cv2.waitKey(1)
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()


0: 640x512 3 persons, 9.3ms
Error: OpenCV(4.9.0) /io/opencv/modules/imgproc/src/resize.cpp:4155: error: (-215:Assertion failed) inv_scale_x > 0 in function 'resize'

Error: OpenCV(4.9.0) /io/opencv/modules/imgproc/src/resize.cpp:4155: error: (-215:Assertion failed) inv_scale_x > 0 in function 'resize'

Error: OpenCV(4.9.0) /io/opencv/modules/imgproc/src/resize.cpp:4155: error: (-215:Assertion failed) inv_scale_x > 0 in function 'resize'

Speed: 2.1ms preprocess, 9.3ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 512)

0: 640x512 1 person, 7.8ms
Error: OpenCV(4.9.0) /io/opencv/modules/imgproc/src/resize.cpp:4155: error: (-215:Assertion failed) inv_scale_x > 0 in function 'resize'

Speed: 3.0ms preprocess, 7.8ms inference, 2.7ms postprocess per image at shape (1, 3, 640, 512)

0: 640x512 1 person, 12.1ms
Error: OpenCV(4.9.0) /io/opencv/modules/imgproc/src/resize.cpp:4155: error: (-215:Assertion failed) inv_scale_x > 0 in function 'resize'

Speed: 4.5ms preprocess, 12.1ms 

In [15]:
from ultralytics import YOLO
import cv2
import numpy as np
import os
import time

# YOLO 모델 로드
model = YOLO('yolov8n-pose.pt')
cap = cv2.VideoCapture(0)

# 이미지 파일 목록 로드
image_folder = '/home/min/dev_ws/yolo/data'
image_files = [os.path.join(image_folder, f) for f in os.listdir(image_folder)]
image_index = 0


# 동물 얼굴 이미지 로드
img2_path = image_files[image_index]
img2 = cv2.imread(img2_path)

# 메시지 및 타이머 변수
message = "if you want to save the picture, press the 's'"
last_saved_time = None

def apply_animal_face(frame, keypoints, img2):
    try:
        org = keypoints[0]
        right_ear = keypoints[3]
        left_ear = keypoints[4]

        # 동물 얼굴 이미지 크기 조정
        resize_cal = int(abs(right_ear[0] - left_ear[0]) * 1.4)
        img2_resized = cv2.resize(img2, (resize_cal, resize_cal), interpolation=cv2.INTER_LINEAR)
        img2H, img2W, _ = img2_resized.shape

        # 위치 계산
        x_offset = org[0] - img2W // 2
        y_offset = org[1] - int(img2H * 0.7)

        # ROI 설정
        roi = frame[y_offset:y_offset + img2H, x_offset:x_offset + img2W]

        # 동물 얼굴 이미지 그레이스케일 및 마스크 생성
        img2_gray = cv2.cvtColor(img2_resized, cv2.COLOR_BGR2GRAY)
        mask_inv = cv2.bitwise_not(img2_gray)
        fg = cv2.bitwise_or(img2_resized, img2_resized, mask=mask_inv)

        # ROI와 동물 얼굴 이미지 결합
        final_roi = cv2.bitwise_or(roi, fg)
        frame[y_offset:y_offset + img2H, x_offset:x_offset + img2W] = final_roi

    except Exception as e:
        print(f"Error: {e}")

save_folder = 'saved_images'

if not os.path.exists(save_folder):
    os.makedirs(save_folder)

while True:
    ret, frame = cap.read()
    if not ret:
        print('ERROR: frame error')
        break

    frame = cv2.resize(frame, (720, 900))
    results = model(frame, stream=True)

    for r in results:
        keypoints = r.keypoints

        for ks in keypoints:
            k = np.array(ks.xy[0].cpu(), dtype=int)
            apply_animal_face(frame, k, img2)

    # 현재 시간 확인
    current_time = time.time()

    # 2초가 지나면 메시지를 원래대로 되돌림
    if last_saved_time and (current_time - last_saved_time) > 2:
        message = "if you want to save the picture, press the 's'"
        last_saved_time = None

    # 프레임에 텍스트 추가
    cv2.putText(frame, message, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)


    cv2.imshow('result preview', frame)
    key = cv2.waitKey(1)
    if key == 27:
        break
    elif key == ord(' '):
        # 스페이스바 눌렀을 때 이미지 변경
        image_index = (image_index + 1) % len(image_files)
        img2_path = image_files[image_index]
        img2 = cv2.imread(img2_path)

    elif key == ord('s'):
        # 's' 키가 눌렸을 때 이미지 저장
        img2_filename = os.path.basename(img2_path)
        save_filename = f'captured_{img2_filename}'
        save_path = os.path.join(save_folder, save_filename)
        cv2.imwrite(save_path, frame)
        print(f"Image saved to {save_path}")

        # 메시지를 변경하고 타이머 시작
        message = "picture has been saved"
        last_saved_time = time.time()

cap.release()
cv2.destroyAllWindows()

ERROR: frame error


[ WARN:0@2431.073] global cap_v4l.cpp:997 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@2431.073] global obsensor_uvc_stream_channel.cpp:159 getStreamChannelGroup Camera index out of range
